In [1]:
#!pip install unsloth

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

2025-03-27 08:29:45.607313: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-27 08:29:45.617800: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743053385.629260   16856 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743053385.632574   16856 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-27 08:29:45.644785: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/SmolLM2-1.7B", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B Unsloth-SmolLM2-360M-Lora
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code=True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.584 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0.2, # Supports any, but = 0 is optimized
    bias = "all",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config ={"bits": 4}, # And LoftQ
    init_lora_weights="loftq",
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.2.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth: bias = `none` is supported for fast patching. You are using bias = all.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.2.15 patched 24 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [6]:

from datasets import load_dataset
# 2. Tapaco Veri Setini Yükleyin

dataset = Dataset.from_csv("BKÜ Sınav Analizi - BKÜ Sınav Analizi.csv")
dataset_test = Dataset.from_csv("tarim_veriseti_test.csv")

In [7]:
dataset = dataset.filter(lambda x: x["GPT"]!=None and x["Soru"]!=None)
dataset = dataset.filter(lambda x: len(x["GPT"])<max_seq_length)

In [8]:
dataset

Dataset({
    features: ['Soru', 'Cevap', 'Doğru Cevap', 'GPT', 'inputs'],
    num_rows: 16350
})

In [9]:
dataset_test


Dataset({
    features: ['Soru', 'Cevap'],
    num_rows: 100
})

In [10]:
dataset = dataset.rename_columns({"Soru": "instruction", "GPT": "output"})
dataset_test = dataset_test.rename_columns({"Soru": "instruction", "Cevap": "output"})

In [11]:
from datasets import Dataset
import numpy as np

# None içeren satırları temizleyen fonksiyon
def remove_none_rows(example):
    return example["instruction"] is not None and example["output"] is not None

# None değerleri içeren satırları filtrele
dataset = dataset.filter(remove_none_rows)

In [12]:
system_prompt_text = """
Senin adın Crispy. Sen bir tarım uzmanı ve ziraat mühendisisin. Kullanıcılara yalnızca tarım, bitki hastalıkları, tarımsal ilaçlar, tarım ekipmanları, sulama sistemleri, gübreleme teknikleri ve çiftçilikle ilgili konularda teknik ve detaylı bilgiler sağlarsın.

Başka bir konu sorulduğunda, yalnızca tarımla ilgili konular hakkında yardımcı olabileceğini nazikçe belirtmelisin.

### Tarım ile ilgili anahtar konular:
- **Bitki hastalıkları ve zararlılar:** Mantar hastalıkları, bakteriyel hastalıklar, virüsler, böcekler, fungusitler, herbisitler, insektisitler.
- **Toprak ve Gübreleme:** Organik ve kimyasal gübreler, toprak analizi, sürdürülebilir tarım yöntemleri, pH dengesi.
- **Sulama Sistemleri:** Damla sulama, pivot sulama, yağmurlama sulama, sulama suyu yönetimi.
- **Tarım Makineleri ve Mekanizasyon:** Traktörler, biçerdöverler, ekim makineleri, hasat makineleri, modern tarım teknolojileri.
- **Sebzecilik ve Meyvecilik:** Açık tarla üretimi, seracılık, budama, tozlanma, meyve ağaçlarının bakım süreçleri.
- **Tahıl ve Baklagiller Üretimi:** Buğday, arpa, mısır, mercimek, nohut gibi tarla bitkileri.
- **Tarımsal Biyoteknoloji:** Genetiği değiştirilmiş organizmalar (GDO), bitki ıslahı, verim artırma teknikleri.
- **İklim ve Tarım:** Kuraklık yönetimi, iklim değişikliğinin tarıma etkileri, su kaynakları yönetimi.

### Tarım Dışı Konular İçin Yanıt:
Eğer kullanıcı tarımla ilgili olmayan bir konu sorarsa, şu şekilde yanıt vermelisin:
*"Ben Crispy, bir tarım uzmanıyım. Yalnızca tarım ve ziraat konularında yardımcı olabilirim."*

### Selamlaşma ve Etkileşim:
- Kullanıcı "Merhaba" veya "Nasılsın?" gibi ifadeler kullandığında, dostça yanıt ver.
  - Örneğin: *"Merhaba! Ben Crispy, tarım konusunda sana yardımcı olmaya hazırım. Bugün hangi konuda bilgi almak istersin?"*
- Kullanıcı veda ettiğinde, uygun şekilde yanıt ver.
  - Örneğin: *"Görüşmek üzere! Tarımla ilgili her zaman bana danışabilirsin."*
"""


In [13]:
alpaca_prompt = """{}

### Talimat:
{}

### Giriş:
{}

### Yanıt:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):

    instructions = examples["instruction"]

    inputs       = examples["inputs"]
    outputs      = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        input = instruction.split("?")[1] if "?" in instruction else ""
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(system_prompt_text,instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass



In [14]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [15]:
dataset

Dataset({
    features: ['instruction', 'Cevap', 'Doğru Cevap', 'output', 'inputs', 'text'],
    num_rows: 16350
})

In [16]:
dataset = dataset.shuffle(seed=42)
dataset = dataset.shuffle(seed=41)
dataset = dataset.shuffle(seed=40)
dataset = dataset.shuffle(seed=39)

In [17]:
dataset

Dataset({
    features: ['instruction', 'Cevap', 'Doğru Cevap', 'output', 'inputs', 'text'],
    num_rows: 16350
})

In [18]:
print(dataset[35]["text"])


Senin adın Crispy. Sen bir tarım uzmanı ve ziraat mühendisisin. Kullanıcılara yalnızca tarım, bitki hastalıkları, tarımsal ilaçlar, tarım ekipmanları, sulama sistemleri, gübreleme teknikleri ve çiftçilikle ilgili konularda teknik ve detaylı bilgiler sağlarsın.

Başka bir konu sorulduğunda, yalnızca tarımla ilgili konular hakkında yardımcı olabileceğini nazikçe belirtmelisin.

### Tarım ile ilgili anahtar konular:
- **Bitki hastalıkları ve zararlılar:** Mantar hastalıkları, bakteriyel hastalıklar, virüsler, böcekler, fungusitler, herbisitler, insektisitler.
- **Toprak ve Gübreleme:** Organik ve kimyasal gübreler, toprak analizi, sürdürülebilir tarım yöntemleri, pH dengesi.
- **Sulama Sistemleri:** Damla sulama, pivot sulama, yağmurlama sulama, sulama suyu yönetimi.
- **Tarım Makineleri ve Mekanizasyon:** Traktörler, biçerdöverler, ekim makineleri, hasat makineleri, modern tarım teknolojileri.
- **Sebzecilik ve Meyvecilik:** Açık tarla üretimi, seracılık, budama, tozlanma, meyve ağaçlar

In [19]:
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2, seed=42).values()


In [20]:
import wandb

wb_c = wandb.init(project="Basic LLM Train", name="SmolLM2-1.7B-R32-Bias-DO",  resume="allow", id="t66dqcif") #id="a7zeymst",id="ecibz7e4" id="dbaxrwf4"
wb_c.watch(model, log="all")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: h-osmankarabulut. Use `wandb login --relogin` to force relogin


In [21]:
import wandb
import evaluate
from transformers import TrainerCallback
import gc 


class WandbRougeCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.rouge = evaluate.load("rouge")
        self.test_dataset = test_dataset
        self.epoch_interval = 1  # Her 3 epoch'ta bir işlem yapmak için ayar
        self.current_epoch = 0


        
    
    def on_train_end(self, args, state, control, **kwargs):
            
        self.current_epoch += 1
        # Her 3. epoch'ta işlem yap
        if self.current_epoch % self.epoch_interval == 0:
            
            print(f"Epoch {self.current_epoch} - Doing something special!")

            model = kwargs.get("model", None)
            tokenizer = kwargs.get("tokenizer", None)
            FastLanguageModel.for_inference(model)
            if model is None or tokenizer is None:
                return
            
            predictions = []
            references = []
            
            for example in self.test_dataset:
                input_text = alpaca_prompt.format(system_prompt_text,example["instruction"], input, "") + EOS_TOKEN  
                reference_text = example["output"]
                inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
                output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
                decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
                
                decoded_output = decoded_output.split("### Yanıt")[1]
                
                predictions.append(decoded_output)
                references.append(reference_text)
            
            scores = self.rouge.compute(predictions=predictions, references=references)
            
            wb_c.log({
                "ROUGE-1": scores["rouge1"],
                "ROUGE-2": scores["rouge2"],
                "ROUGE-L": scores["rougeL"]
            })

            #del predictions, references, decoded_output, scores, output_ids, reference_text, input_text, model, tokenizer, self.rouge 

            # GPU bellek temizleme
            #torch.cuda.empty_cache()  # Kullanılmayan GPU belleğini boşalt
            #gc.collect()  # Python çöp toplama çalıştır
   

class WandbBleuCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.bleu = evaluate.load("bleu")
        self.test_dataset = test_dataset
        self.epoch_interval = 1  # Her 3 epoch'ta bir işlem yapmak için ayar
        self.current_epoch = 0
        
        
        
    
    def on_train_end(self, args, state, control, **kwargs):
            
        self.current_epoch += 1
        # Her 3. epoch'ta işlem yap
        if self.current_epoch % self.epoch_interval == 0:
            print(f"Epoch {self.current_epoch} - Doing something special!")

            model = kwargs.get("model", None)
            tokenizer = kwargs.get("tokenizer", None)
            FastLanguageModel.for_inference(model)
            if model is None or tokenizer is None:
                return
            
            predictions = []
            references = []
            
            for example in self.test_dataset:
                input_text = alpaca_prompt.format(system_prompt_text,example["instruction"], input, "") + EOS_TOKEN  
                reference_text = example["output"]
                inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
                output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
                decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
                
                decoded_output = decoded_output.split("### Yanıt")[1]

                predictions.append(decoded_output)
                references.append(reference_text)
            
            scores = self.bleu.compute(predictions=predictions, references=[[ref] for ref in references])
            
            wb_c.log({
                "BLEU": scores["bleu"]
            })

            #del predictions, references, decoded_output, scores, output_ids, reference_text, input_text, model, tokenizer, self.bleu

            # GPU bellek temizleme
            #torch.cuda.empty_cache()  # Kullanılmayan GPU belleğini boşalt
            #gc.collect()  # Python çöp toplama çalıştır
        

class WandbMeteorCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.meteor = evaluate.load("meteor")
        self.test_dataset = test_dataset
        self.epoch_interval = 1  # Her 3 epoch'ta bir işlem yapmak için ayar
        self.current_epoch = 0
        
       
        
    
    def on_train_end(self, args, state, control, **kwargs):
            
        self.current_epoch += 1
        # Her 3. epoch'ta işlem yap
        if self.current_epoch % self.epoch_interval == 0:
            print(f"Epoch {self.current_epoch} - Doing something special!")

            model = kwargs.get("model", None)
            tokenizer = kwargs.get("tokenizer", None)
            FastLanguageModel.for_inference(model)
            if model is None or tokenizer is None:
                return
            
            predictions = []
            references = []
            
            for example in self.test_dataset:
                input_text = alpaca_prompt.format(system_prompt_text,example["instruction"], input, "") + EOS_TOKEN  
                reference_text = example["output"]
                inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
                output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
                decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
                
                decoded_output = decoded_output.split("### Yanıt")[1]

                predictions.append(decoded_output)
                references.append(reference_text)
            
            scores = self.meteor.compute(predictions=predictions, references=references)
            
            wb_c.log({
                "METEOR": scores["meteor"]
            })

            #del predictions, references, decoded_output, scores, output_ids, reference_text, input_text, model, tokenizer, self.meteor

            # GPU bellek temizleme
            #torch.cuda.empty_cache()  # Kullanılmayan GPU belleğini boşalt
            #gc.collect()  # Python çöp toplama çalıştır
    
    
class WandbBertScoreCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.bertscore = evaluate.load("bertscore")
        self.test_dataset = test_dataset
        self.epoch_interval = 1  # Her 3 epoch'ta bir işlem yapmak için ayar
        self.current_epoch = 0
        
       
        
    
    def on_train_end(self, args, state, control, **kwargs):
        
        self.current_epoch += 1
        # Her 3. epoch'ta işlem yap
        if self.current_epoch % self.epoch_interval == 0:
            print(f"Epoch {self.current_epoch} - Doing something special!")

            model = kwargs.get("model", None)
            tokenizer = kwargs.get("tokenizer", None)
            FastLanguageModel.for_inference(model)
            if model is None or tokenizer is None:
                return
            
            predictions = []
            references = []
            
            for example in self.test_dataset:
                input_text = alpaca_prompt.format(system_prompt_text,example["instruction"], input, "") + EOS_TOKEN  
                reference_text = example["output"]
                inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
                output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
                decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

                decoded_output = decoded_output.split("### Yanıt")[1]

                predictions.append(decoded_output)
                references.append(reference_text)
            
            scores = self.bertscore.compute(predictions=predictions, references=references, lang="tr")
            
            wb_c.log({
            "BERTScore Precision": np.mean(scores["precision"]),
            "BERTScore Recall": np.mean(scores["recall"]),
            "BERTScore F1": np.mean(scores["f1"])
        })  
            
            #del predictions, references, decoded_output, scores, output_ids, reference_text, input_text, model, tokenizer, self.bertscore

            # GPU bellek temizleme
            #torch.cuda.empty_cache()  # Kullanılmayan GPU belleğini boşalt
            #gc.collect()  # Python çöp toplama çalıştır
    
    


In [22]:
# FastLanguageModel.for_inference(model)
# example = dataset_test[18]
    
# input_text = alpaca_prompt.format(system_prompt_text,example["instruction"], input, "") + EOS_TOKEN  
# reference_text = example["output"]
# inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
# output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
# decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True) 

In [23]:
#example["instruction"]

In [24]:
#input_text

In [25]:
#decoded_output

In [26]:
#"### Yanıt"+decoded_output.split("### Yanıt")[1]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    #data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    #callbacks=[wandb_callback],
    args = TrainingArguments(
       
        gradient_accumulation_steps = 1,
        num_train_epochs=4,  
        per_device_train_batch_size=16,       # GPU başına batch boyutu
        per_device_eval_batch_size=16,       # GPU başına batch boyutu
        learning_rate = 0.001,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 300,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        eval_steps=500,
        evaluation_strategy="steps",
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "SmolLM2-1.7B-R32-Bias-DO",
        report_to="wandb",                    # WandB veya diğer araçlara raporlama yok
        save_total_limit=2,                  # Sadece son iki checkpoint'i sakla
        save_steps=300,
        warmup_steps=500,           # İlk 1000 adımda LR'yi yavaş yavaş artır
        
    ),
)

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Tokenizing train dataset (num_proc=2):   0%|          | 0/13080 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/13080 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/3270 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/3270 [00:00<?, ? examples/s]

In [28]:
# trainer.add_callback(WandbRougeCallback(dataset_test))
# trainer.add_callback(WandbBleuCallback(dataset_test))
# trainer.add_callback(WandbMeteorCallback(dataset_test))
# trainer.add_callback(WandbBertScoreCallback(dataset_test)) 

In [29]:
wandb.watch(model, log="all")

In [30]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 13,080 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 1
\        /    Total batch size = 16 | Total steps = 3,272
 "-____-"     Number of trainable parameters = 36,175,872
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/3272 [00:00<?, ?it/s]

Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


{'loss': 0.3655, 'grad_norm': 0.04250284284353256, 'learning_rate': 0.0006, 'epoch': 0.37}


  0%|          | 0/205 [00:00<?, ?it/s]

...

{'eval_loss': 0.03858543559908867, 'eval_runtime': 399.9502, 'eval_samples_per_second': 8.176, 'eval_steps_per_second': 0.513, 'epoch': 2.93}
{'loss': 0.0285, 'grad_norm': 0.020527269691228867, 'learning_rate': 0.00020634920634920634, 'epoch': 3.3}


  0%|          | 0/205 [00:00<?, ?it/s]

{'eval_loss': 0.039371997117996216, 'eval_runtime': 387.6793, 'eval_samples_per_second': 8.435, 'eval_steps_per_second': 0.529, 'epoch': 3.3}
{'loss': 0.0295, 'grad_norm': 0.012711185961961746, 'learning_rate': 9.812409812409813e-05, 'epoch': 3.67}


  0%|          | 0/205 [00:00<?, ?it/s]

{'eval_loss': 0.03922688961029053, 'eval_runtime': 403.7588, 'eval_samples_per_second': 8.099, 'eval_steps_per_second': 0.508, 'epoch': 3.67}
{'train_runtime': 20714.5336, 'train_samples_per_second': 2.526, 'train_steps_per_second': 0.158, 'train_loss': 0.06869891322329458, 'epoch': 4.0}


In [31]:
trainer.evaluate()

  0%|          | 0/205 [00:00<?, ?it/s]

{'eval_loss': 0.03817741200327873,
 'eval_runtime': 355.7342,
 'eval_samples_per_second': 9.192,
 'eval_steps_per_second': 0.576}

In [32]:
import wandb
import evaluate
import numpy as np
from prettytable import PrettyTable
import torch

def evaluate_model(model, tokenizer, test_dataset, max_seq_length=256):
    """
    Eğitilmiş modeli test veri kümesi üzerinde değerlendirir ve sonuçları wandb'a loglar.
    
    Parametreler:
    - model: Eğitilmiş dil modeli
    - tokenizer: Modelin tokenizer'ı
    - test_dataset: Test veri kümesi (instruction-output içermeli)
    - max_seq_length: Maksimum yanıt uzunluğu (varsayılan: 256)

    Çıktı:
    - Metin tablosu (PrettyTable ile)
    - wandb logları
    """
    
    # Değerlendirme metriklerini yükleme
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")
    meteor = evaluate.load("meteor")
    bertscore = evaluate.load("bertscore")

    predictions = []
    references = []

    # Modeli değerlendirme moduna al
    model.eval()

    print("🚀 Model test verisi üzerinde değerlendiriliyor...\n")

    for example in test_dataset:
        input_text = f"### Talimat:\n{example['instruction']}\n\n### Yanıt:\n"
        reference_text = example["output"]

        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

        with torch.no_grad():
            output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
        
        decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Eğer yanıt formatında başlıklar varsa temizleyelim
        decoded_output = decoded_output.split("### Yanıt")[-1].strip()

        predictions.append(decoded_output)
        references.append(reference_text)

    # Metrik hesaplamaları
    rouge_scores = rouge.compute(predictions=predictions, references=references)
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    meteor_score = meteor.compute(predictions=predictions, references=references)
    bert_scores = bertscore.compute(predictions=predictions, references=references, lang="tr")

    # BERTScore ortalamaları al
    bert_precision = np.mean(bert_scores["precision"])
    bert_recall = np.mean(bert_scores["recall"])
    bert_f1 = np.mean(bert_scores["f1"])

    # Sonuçları tabloya ekle
    table = PrettyTable()
    table.field_names = ["Metrik", "Değer"]
    table.add_row(["ROUGE-1", round(rouge_scores["rouge1"], 4)])
    table.add_row(["ROUGE-2", round(rouge_scores["rouge2"], 4)])
    table.add_row(["ROUGE-L", round(rouge_scores["rougeL"], 4)])
    table.add_row(["BLEU", round(bleu_score["bleu"], 4)])
    table.add_row(["METEOR", round(meteor_score["meteor"], 4)])
    table.add_row(["BERTScore Precision", round(bert_precision, 4)])
    table.add_row(["BERTScore Recall", round(bert_recall, 4)])
    table.add_row(["BERTScore F1", round(bert_f1, 4)])

    # Sonuçları yazdır
    print(table)

    # WandB logları
    wandb.log({
        "ROUGE-1": rouge_scores["rouge1"],
        "ROUGE-2": rouge_scores["rouge2"],
        "ROUGE-L": rouge_scores["rougeL"],
        "BLEU": bleu_score["bleu"],
        "METEOR": meteor_score["meteor"],
        "BERTScore Precision": bert_precision,
        "BERTScore Recall": bert_recall,
        "BERTScore F1": bert_f1
    })

    print("\n✅ Model değerlendirme tamamlandı ve sonuçlar wandb'a loglandı.")


In [34]:
FastLanguageModel.for_inference(model)

# Test değerlendirmesi
evaluate_model(model, tokenizer, dataset_test)

[nltk_data] Downloading package wordnet to /home/hosman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/hosman/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/hosman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


🚀 Model test verisi üzerinde değerlendiriliyor...



tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

+---------------------+--------+
|        Metrik       | Değer  |
+---------------------+--------+
|       ROUGE-1       | 0.2439 |
|       ROUGE-2       | 0.1303 |
|       ROUGE-L       | 0.2147 |
|         BLEU        | 0.0406 |
|        METEOR       | 0.2262 |
| BERTScore Precision | 0.5286 |
|   BERTScore Recall  | 0.5834 |
|     BERTScore F1    | 0.553  |
+---------------------+--------+

✅ Model değerlendirme tamamlandı ve sonuçlar wandb'a loglandı.


In [35]:
wb_c.finish()

BERTScore F1,▁
BERTScore Precision,▁
BERTScore Recall,▁
BLEU,▁
METEOR,▁
ROUGE-1,▁
ROUGE-2,▁
ROUGE-L,▁
eval/loss,█▅▃▂▂▂▁▁▁▁▁
eval/runtime,▆█▅▇▆▇▆▇▆█▁
eval/samples_per_second,▃▁▃▂▃▂▃▁▃▁█


In [36]:
model.save_pretrained("SmolLM2-1.7B-R32-Bias-DO") # Local saving
tokenizer.save_pretrained("SmolLM2-1.7B-R32-Bias-DO")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('SmolLM2-1.7B-R32-Bias-DO/tokenizer_config.json',
 'SmolLM2-1.7B-R32-Bias-DO/special_tokens_map.json',
 'SmolLM2-1.7B-R32-Bias-DO/vocab.json',
 'SmolLM2-1.7B-R32-Bias-DO/merges.txt',
 'SmolLM2-1.7B-R32-Bias-DO/added_tokens.json',
 'SmolLM2-1.7B-R32-Bias-DO/tokenizer.json')

INFERENCE

In [37]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.



In [38]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "SmolLM2-1.7B-R32-Bias-DO",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code=True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.584 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [39]:
system_prompt_text = """
Senin adın Crispy. Sen bir tarım uzmanı ve ziraat mühendisisin. Kullanıcılara yalnızca tarım, bitki hastalıkları, tarımsal ilaçlar, tarım ekipmanları, sulama sistemleri, gübreleme teknikleri ve çiftçilikle ilgili konularda teknik ve detaylı bilgiler sağlarsın.

Başka bir konu sorulduğunda, yalnızca tarımla ilgili konular hakkında yardımcı olabileceğini nazikçe belirtmelisin.

### Tarım ile ilgili anahtar konular:
- **Bitki hastalıkları ve zararlılar:** Mantar hastalıkları, bakteriyel hastalıklar, virüsler, böcekler, fungusitler, herbisitler, insektisitler.
- **Toprak ve Gübreleme:** Organik ve kimyasal gübreler, toprak analizi, sürdürülebilir tarım yöntemleri, pH dengesi.
- **Sulama Sistemleri:** Damla sulama, pivot sulama, yağmurlama sulama, sulama suyu yönetimi.
- **Tarım Makineleri ve Mekanizasyon:** Traktörler, biçerdöverler, ekim makineleri, hasat makineleri, modern tarım teknolojileri.
- **Sebzecilik ve Meyvecilik:** Açık tarla üretimi, seracılık, budama, tozlanma, meyve ağaçlarının bakım süreçleri.
- **Tahıl ve Baklagiller Üretimi:** Buğday, arpa, mısır, mercimek, nohut gibi tarla bitkileri.
- **Tarımsal Biyoteknoloji:** Genetiği değiştirilmiş organizmalar (GDO), bitki ıslahı, verim artırma teknikleri.
- **İklim ve Tarım:** Kuraklık yönetimi, iklim değişikliğinin tarıma etkileri, su kaynakları yönetimi.

### Tarım Dışı Konular İçin Yanıt:
Eğer kullanıcı tarımla ilgili olmayan bir konu sorarsa, şu şekilde yanıt vermelisin:
*"Ben Crispy, bir tarım uzmanıyım. Yalnızca tarım ve ziraat konularında yardımcı olabilirim."*

### Selamlaşma ve Etkileşim:
- Kullanıcı "Merhaba" veya "Nasılsın?" gibi ifadeler kullandığında, dostça yanıt ver.
  - Örneğin: *"Merhaba! Ben Crispy, tarım konusunda sana yardımcı olmaya hazırım. Bugün hangi konuda bilgi almak istersin?"*
- Kullanıcı veda ettiğinde, uygun şekilde yanıt ver.
  - Örneğin: *"Görüşmek üzere! Tarımla ilgili her zaman bana danışabilirsin."*
"""


In [40]:
alpaca_prompt = """{}

### Talimat:
{}

### Giriş:
{}

### Yanıt:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):

    instructions = examples["instruction"]

    inputs       = examples["inputs"]
    outputs      = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        input = instruction.split("?")[1] if "?" in instruction else ""
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(system_prompt_text,instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass



In [41]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 32,
#     lora_dropout = 0.2, # Supports any, but = 0 is optimized
#     bias = "all",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config ={"bits": 4}, # And LoftQ
#     init_lora_weights="loftq",
# )

In [52]:
from unsloth.chat_templates import get_chat_template

# Tokenizer'a chat şablonunu uygula
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",  # Desteklenen formatlar: zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping={
        "role": "from",
        "content": "value",
        "user": "human",
        "assistant": "gpt"
    },  # ShareGPT tarzı eşleme
    map_eos_token=True  # <|im_end|> ifadesini </s> ile eşle
)

# Modeli çıkarım (inference) için hazırla
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    alpaca_prompt.format(
        system_prompt_text,
        "Avokado hangi iklimde yetişir ve nasıl yetiştirilmelidir?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")



# Modelden yanıt üret
outputs = model.generate(
    **inputs,
    max_new_tokens=2048,
    use_cache=True
)

# Yanıtları çözümle
decoded_outputs = tokenizer.batch_decode(outputs)

print(decoded_outputs)

# Çıktıyı formatlayan fonksiyon
def format_chat_output(decoded_outputs):
    formatted_text = decoded_outputs[0].replace("<|im_start|>user\n", "🗣 **Kullanıcı:**\n") \
                                      .replace("<|im_start|>assistant\n", "🤖 **Asistan:**\n") \
                                      .replace("<|im_end|>", "").strip()
    return formatted_text

# Formatlanmış çıktıyı ekrana yazdır
print(format_chat_output(decoded_outputs))


['\nSenin adın Crispy. Sen bir tarım uzmanı ve ziraat mühendisisin. Kullanıcılara yalnızca tarım, bitki hastalıkları, tarımsal ilaçlar, tarım ekipmanları, sulama sistemleri, gübreleme teknikleri ve çiftçilikle ilgili konularda teknik ve detaylı bilgiler sağlarsın.\n\nBaşka bir konu sorulduğunda, yalnızca tarımla ilgili konular hakkında yardımcı olabileceğini nazikçe belirtmelisin.\n\n### Tarım ile ilgili anahtar konular:\n- **Bitki hastalıkları ve zararlılar:** Mantar hastalıkları, bakteriyel hastalıklar, virüsler, böcekler, fungusitler, herbisitler, insektisitler.\n- **Toprak ve Gübreleme:** Organik ve kimyasal gübreler, toprak analizi, sürdürülebilir tarım yöntemleri, pH dengesi.\n- **Sulama Sistemleri:** Damla sulama, pivot sulama, yağmurlama sulama, sulama suyu yönetimi.\n- **Tarım Makineleri ve Mekanizasyon:** Traktörler, biçerdöverler, ekim makineleri, hasat makineleri, modern tarım teknolojileri.\n- **Sebzecilik ve Meyvecilik:** Açık tarla üretimi, seracılık, budama, tozlanma, m

adapter_model.safetensors:   0%|          | 0.00/72.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hosmankarabulut/SmolLM2-Ziraat-Turkish-v1/commit/20b63ef02b6630c46a3bc30c626dbef3e698f1e4', commit_message='Initial upload of Turkish Ziraat fine-tuned model', commit_description='', oid='20b63ef02b6630c46a3bc30c626dbef3e698f1e4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hosmankarabulut/SmolLM2-Ziraat-Turkish-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='hosmankarabulut/SmolLM2-Ziraat-Turkish-v1'), pr_revision=None, pr_num=None)

In [43]:
""" import pandas as pd
import torch
import random
from unsloth.chat_templates import get_chat_template
from difflib import SequenceMatcher
from tqdm import tqdm

# GPU kullanımı kontrol et
device = "cuda" if torch.cuda.is_available() else "cpu"

# Chat şablonunu Tokenizer'a uygula
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    mapping={
        "role": "from",
        "content": "value",
        "user": "human",
        "assistant": "gpt"
    },
    map_eos_token=True
)

# Modeli çıkarım (inference) için hazırla
FastLanguageModel.for_inference(model)
model.to(device)  # GPU'ya taşı

# CSV dosyasını oku
csv_file = "BKÜ Sınav Analizi.csv"
df = pd.read_csv(csv_file)

# Veriyi karıştır ve sadece %25'ini kullan
df = df.sample(frac=1, random_state=42)  # Karıştır
df = df.sample(frac=0.99, random_state=42)  # %25'ini seç

# Doğru tahminleri saymak için sayaç
correct_count = 0
total_questions = len(df)

# Benzerlik hesaplayan fonksiyon
def similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

# Soruları tek tek modele gönder ve doğruluğu ölç
for index, row in tqdm(df.iterrows(), total=len(df)):
    question = row["Soru"]
    correct_answer = row["GPT"]

    messages = [{"from": "human", "value": question}]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(device)

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=512,
        use_cache=True
    )

    decoded_outputs = tokenizer.batch_decode(outputs)
    
    # Model çıktısını formatla
    model_answer = decoded_outputs[0].replace("<|im_start|>user\n", "").replace("<|im_start|>assistant\n", "").replace("<|im_end|>", "").strip()
    model_answer = model_answer[len(question):]

    # Benzerlik oranını hesapla
    match_ratio = similarity(str(model_answer), str(correct_answer))

    # %80'den büyükse doğru kabul et
    if match_ratio > 0.8:
        correct_count += 1

    #print(f"Soru: {question}")
    #print(f"Model Cevabı: {model_answer}")
    #print(f"Gerçek Cevap: {correct_answer}")
    #print(f"Benzerlik: %{match_ratio * 100:.2f}")
   # print("-" * 50)

# Doğruluk yüzdesini hesapla
accuracy = (correct_count / total_questions) * 100
print(f"Modelin doğruluk oranı: %{accuracy:.2f}")
 """

' import pandas as pd\nimport torch\nimport random\nfrom unsloth.chat_templates import get_chat_template\nfrom difflib import SequenceMatcher\nfrom tqdm import tqdm\n\n# GPU kullanımı kontrol et\ndevice = "cuda" if torch.cuda.is_available() else "cpu"\n\n# Chat şablonunu Tokenizer\'a uygula\ntokenizer = get_chat_template(\n    tokenizer,\n    chat_template="chatml",\n    mapping={\n        "role": "from",\n        "content": "value",\n        "user": "human",\n        "assistant": "gpt"\n    },\n    map_eos_token=True\n)\n\n# Modeli çıkarım (inference) için hazırla\nFastLanguageModel.for_inference(model)\nmodel.to(device)  # GPU\'ya taşı\n\n# CSV dosyasını oku\ncsv_file = "BKÜ Sınav Analizi.csv"\ndf = pd.read_csv(csv_file)\n\n# Veriyi karıştır ve sadece %25\'ini kullan\ndf = df.sample(frac=1, random_state=42)  # Karıştır\ndf = df.sample(frac=0.99, random_state=42)  # %25\'ini seç\n\n# Doğru tahminleri saymak için sayaç\ncorrect_count = 0\ntotal_questions = len(df)\n\n# Benzerlik hesapl